In [ ]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [ ]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized


In [ ]:
# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

[<Model Human-GEM_28062023 at 0x1d54cf89510>,
 <Model mod_RPE_PRcone_Liang at 0x1d555c4cbb0>,
 <Model mod_RPE_PRcone_Liang_singlePR at 0x1d520e0f790>,
 <Model mod_RPE_PRcone_Liang_singleRPE at 0x1d523493130>,
 <Model mod_RPE_PRcone_Lukowski at 0x1d5312bfbb0>,
 <Model mod_RPE_PRcone_Lukowski_singlePR at 0x1d53b00b460>,
 <Model mod_RPE_PRcone_Lukowski_singleRPE at 0x1d53f38aaa0>,
 <Model mod_RPE_PRcontrol at 0x1d53a9446a0>,
 <Model mod_RPE_PRrod_Liang at 0x1d53aa48580>,
 <Model mod_RPE_PRrod_Liang_singlePR at 0x1d598e0fe20>,
 <Model mod_RPE_PRrod_Liang_singleRPE at 0x1d598e25810>,
 <Model mod_RPE_PRrod_Lukowski at 0x1d5e76169e0>,
 <Model mod_RPE_PRrod_Lukowski_singlePR at 0x1d598e29480>,
 <Model mod_RPE_PRrod_Lukowski_singleRPE at 0x1d5f20eee30>,
 <Model Human1_old at 0x1d60e0f3580>,
 <Model RPE_PR_old at 0x1d544dd7bb0>]

In [ ]:
# modify models
# load blood exchange bounds

# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]
Human1_models = [m for m in model_list if 'Human' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)

In [ ]:
# set exchange bounds for blood
from src.modify_model import set_exchange_bounds
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

,max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,24.7325
mod_RPE_PRcone_Liang,23.9000
mod_RPE_PRcone_Liang_singlePR,11.1350
mod_RPE_PRcone_Liang_singleRPE,23.9000
mod_RPE_PRcone_Lukowski,24.7325
mod_RPE_PRcone_Lukowski_singlePR,22.2350
mod_RPE_PRcone_Lukowski_singleRPE,23.9000
mod_RPE_PRcontrol,24.7325
mod_RPE_PRrod_Liang,23.9000
mod_RPE_PRrod_Liang_singlePR,13.9100


In [ ]:
model_list[-5]

Name,mod_RPE_PRrod_Lukowski
Memory address,1d5e76169e0
Number of metabolites,10678
Number of reactions,14592
Number of genes,2188
Number of groups,149
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [ ]:
# This cell is disabled because it takes a long time to run.
# The results are saved in the folder 'results/unlimited_lactate' and can be loaded from there.
# To enable this cell, remove the comment symbol (#) at the beginning of each line

# FLUX VARIABILITY ANALYSIS

# ex_bounds = [{'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)},\
#             {'MAR09048':(0,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)},\
#             {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)}]

# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

# from cobra.flux_analysis import flux_variability_analysis

# fba = {}
# fva = {}
# fva_loopless = {}

# for bounds in ex_bounds:
#     m = model_list[-5]
#     set_exchange_bounds(m, bounds)
#     fba[str(bounds)] = pd.DataFrame(m.optimize().fluxes)
#     print('fba done with bounds: ' + str(bounds) + ' for model: ' + m.id)
#     fva[str(bounds)] = flux_variability_analysis(m, loopless=False)
#     print('fva done with bounds: ' + str(bounds) + ' for model: ' + m.id)
#     fva_loopless[str(bounds)] = flux_variability_analysis(m, loopless=True)
#     print('fva_loopless done with bounds: ' + str(bounds) + ' for model: ' + m.id)

# # pickle fba, fva and fva_loopless
# from pathlib import Path
# # create folder if it does not exist
# Path(Path().cwd() / 'results' / 'unlimited_lactate').mkdir(parents=True, exist_ok=True)

# import pickle
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fba.pkl', 'wb') as f:
#     pickle.dump(fba, f)
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva.pkl', 'wb') as f:
#     pickle.dump(fva, f)
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva_loopless.pkl', 'wb') as f:
#     pickle.dump(fva_loopless, f)

In [ ]:
# Load FBA and FVA results for unlimited lactate production (output from cell above)

import pickle
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fba.pkl', 'rb') as f:
    fba = pickle.load(f)
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva.pkl', 'rb') as f:
    fva = pickle.load(f)
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva_loopless.pkl', 'rb') as f:
    fva_loopless = pickle.load(f)   

In [ ]:
# fba
df_fba = pd.concat(fba, axis=1)
df_fba = df_fba.rename(columns={'fluxes': 'fba'})

# fva
df_fva = pd.concat(fva, axis=1)
df_fva = df_fva.rename(columns={'minimum': 'min', 'maximum': 'max'})

# loopless fva
df_fva_loopless = pd.concat(fva_loopless, axis=1)
df_fva_loopless = df_fva_loopless.rename(columns={'minimum': 'min_loopless', 'maximum': 'max_loopless'})

# make one big df with fba, fva, loopless fva
df = pd.concat([df_fba, df_fva, df_fva_loopless], axis=1)

# resort columns based on first layer of multiindex per first multi index level
df = df.reindex(sorted(df.columns), axis=1)

# for first layer multiindex
for i in range(len(df.columns.levels[0])):
    # sort second layer multiindex ['fba', 'min', 'max', 'min_loopless', 'max_loopless']
    df = df.reindex(['fba', 'min', 'min_loopless','max','max_loopless'], axis=1, level=1)


{'MAR09048': (-1000, 1000), 'MAR09034': (-4.18, 1000), 'MAR09135': (-1000, 1000)}  \
                                                                                           fba   
MAR03905_RPE                                                0.0                                  
MAR03907_RPE                                                0.0                                  
MAR04281_RPE                                                0.0                                  
MAR04388_RPE                                                0.0                                  
MAR04283_RPE                                                0.0                                  
...                                                         ...                                  
MAR20044_PR                                               400.0                                  
MAR20069_PR                                                 0.0                                  
MAR20070_PR                                                 0.0                                  
MAR20089_PR                                                 0.0                                  
MAR03964_PR                                              1000.0                                  

                                                             \
                 min min_loopless          max max_loopless   
MAR03905_RPE     0.0          0.0     0.000000     0.000000   
MAR03907_RPE     0.0          0.0     0.000000     0.000000   
MAR04281_RPE     0.0          0.0   864.308571   864.308571   
MAR04388_RPE -1000.0      -1000.0  1000.000000     0.000000   
MAR04283_RPE     0.0          0.0     0.000000     0.000000   
...              ...          ...          ...          ...   
MAR20044_PR      0.0          0.0  1000.000000  1000.000000   
MAR20069_PR      0.0          0.0  1000.000000   944.845654   
MAR20070_PR      0.0          0.0  1000.000000     0.000000   
MAR20089_PR      0.0          0.0     0.000000     0.000000   
MAR03964_PR   1000.0       1000.0  1000.000000  1000.000000   

             {'MAR09048': (-3.51, 1000), 'MAR09034': (-4.18, 1000), 'MAR09135': (-1000, 1000)}  \
                                                                                           fba   
MAR03905_RPE                                               0.00                                  
MAR03907_RPE                                               0.00                                  
MAR04281_RPE                                               0.00                                  
MAR04388_RPE                                               0.00                                  
MAR04283_RPE                                               0.00                                  
...                                                         ...                                  
MAR20044_PR                                                7.02                                  
MAR20069_PR                                                0.00                                  
MAR20070_PR                                                0.00                                  
MAR20089_PR                                                0.00                                  
MAR03964_PR                                               25.91                                  

                                                          \
                  min min_loopless      max max_loopless   
MAR03905_RPE     0.00         0.00     0.00         0.00   
MAR03907_RPE     0.00         0.00     0.00         0.00   
MAR04281_RPE     0.00         0.00     0.00         0.00   
MAR04388_RPE -1000.00     -1000.00  1000.00      1000.00   
MAR04283_RPE     0.00         0.00     0.00         0.00   
...               ...          ...      ...          ...   
MAR20044_PR      0.00         0.00  1000.00      1000.00   
MAR20069_PR      0.00         0.00     0.00         0.00   
MAR20070_PR      0.00         0.00     0.00

In [ ]:
# sort rows based on absolute value sizes in min_loopless and max_loopless columns
# get absolute values for loopless columns
# select columns with 'min_loopless' or 'max_loopless' in the second level of the column index
min_max_loopless = df.loc[:, df.columns.get_level_values(1).isin(['min_loopless', 'max_loopless'])]
# transform to absolute values
min_max_loopless = min_max_loopless.abs()
# get sum of absolute values for each row
min_max_loopless['sum'] = min_max_loopless.sum(axis=1)

{'MAR09048': (-1000, 1000), 'MAR09034': (-4.18, 1000), 'MAR09135': (-1000, 1000)}  \
                                                                                              fba   
MAR11276_RPE_PR                                                0.0                                  
MAR00104_PR                                                    0.0                                  
MAR08998_PR                                                    0.0                                  
MAR08997_PR                                                    0.0                                  
MAR04875_PR_RPE                                                0.0                                  
...                                                            ...                                  
MAR04217_RPE                                                   0.0                                  
MAR04230_RPE                                                   0.0                                  
MAR04337_RPE                                                   0.0                                  
MAR04339_RPE                                                   0.0                                  
MAR05444_RPE_PR                                                0.0                                  

                                                           \
                    min min_loopless     max max_loopless   
MAR11276_RPE_PR -1000.0      -1000.0  1000.0       1000.0   
MAR00104_PR     -1000.0      -1000.0  1000.0          0.0   
MAR08998_PR     -1000.0      -1000.0  1000.0       1000.0   
MAR08997_PR     -1000.0      -1000.0  1000.0       1000.0   
MAR04875_PR_RPE -1000.0      -1000.0  1000.0       1000.0   
...                 ...          ...     ...          ...   
MAR04217_RPE        0.0          0.0     0.0          0.0   
MAR04230_RPE        0.0          0.0     0.0          0.0   
MAR04337_RPE        0.0          0.0     0.0          0.0   
MAR04339_RPE        0.0          0.0     0.0          0.0   
MAR05444_RPE_PR     0.0          0.0  1000.0          0.0   

                {'MAR09048': (-3.51, 1000), 'MAR09034': (-4.18, 1000), 'MAR09135': (-1000, 1000)}  \
                                                                                              fba   
MAR11276_RPE_PR                                                0.0                                  
MAR00104_PR                                                    0.0                                  
MAR08998_PR                                                    0.0                                  
MAR08997_PR                                                    0.0                                  
MAR04875_PR_RPE                                                0.0                                  
...                                                            ...                                  
MAR04217_RPE                                                   0.0                                  
MAR04230_RPE                                                   0.0                                  
MAR04337_RPE                                                   0.0                                  
MAR04339_RPE                                                   0.0                                  
MAR05444_RPE_PR                                                0.0                                  

                                                           \
                    min min_loopless     max max_loopless   
MAR11276_RPE_PR -1000.0      -1000.0  1000.0       1000.0   
MAR00104_PR     -1000.0      -1000.0  1000.0       1000.0   
MAR08998_PR     -1000.0      -1000.0  1000.0       1000.0   
MAR08997_PR     -1000.0      -1000.0  1000.0       1000.0   
MAR04875_PR_RPE -1000.0      -1000.0  1000.0       1000.0   
...                 ...          ...     ...          ...   
MAR04217_RPE        0.0          0.0     0.0          0.0   
MAR04230_RPE        0.0          0.0     0.0 

In [ ]:
# make rxn_df
from src.get_info import make_rxn_df
rxn_df = make_rxn_df(model_list[-5])

,cell,lb,ub,bigg.reaction,metanetx.reaction,reactome,tcdb,sbo,ec-code,kegg.reaction,rhea,vmhreaction,name,subsystem,reaction,met_IDs,GPR
MAR03905_RPE,RPE,0.0,1000.0,ALCD2x,MNXR95725,,,SBO:0000176,"[1.1.1.1, 1.1.1.71]",R00754,"[25291, 25290]",ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NAD+[c_RPE] --> acetaldehyde[...,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ENSG00000147576 or ENSG00000172955 or ENSG0000...
MAR03907_RPE,RPE,0.0,1000.0,ALCD2y,MNXR95726,,,SBO:0000176,1.1.1.2,R00746,,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NADP+[c_RPE] --> acetaldehyde...,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ENSG00000117448
MAR04281_RPE,RPE,-1000.0,1000.0,,MNXR101040,,,SBO:0000176,1.1.1.27,R00703,23444,r0173,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[x_RPE] + NADH[x_RPE] + pyruvate[x_RPE] <=> ...,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,ENSG00000111716 or ENSG00000134333
MAR04388_RPE,RPE,-1000.0,1000.0,LDH_L,MNXR101040,,,SBO:0000176,1.1.1.27,R00703,23444,LDH_L,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[c_RPE] + NADH[c_RPE] + pyruvate[c_RPE] <=> ...,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04283_RPE,RPE,0.0,1000.0,,MNXR95750,,,SBO:0000176,1.2.1.5,R00711,"[25299, 25298]",ALDD2y,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,acetaldehyde[c_RPE] + H2O[c_RPE] + NADP+[c_RPE...,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,ENSG00000006534 or ENSG00000108602 or ENSG0000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR20044_PR,PR,0.0,1000.0,,MNXR110840,,,SBO:0000176,1.1.1.284,R07140,19981,,MAR20044,Isolated,hydroxymethylglutathione[c_PR] + NADP+[c_PR] -...,MAM02152c_PR + MAM02554c_PR --> MAM01837c_PR +...,ENSG00000197894
MAR20069_PR,PR,0.0,1000.0,,,,,SBO:0000176,,,,,MAR20069,Glycolysis / Gluconeogenesis,2-(alpha-hydroxyethyl)thiamine-diphosphate[m_P...,MAM00561m_PR + MAM02393m_PR --> MAM02869m_PR +...,(ENSG00000131828 and ENSG00000168291 and ENSG0...
MAR20070_PR,PR,0.0,1000.0,,,,,SBO:0000176,3.1.3.11,,,,MAR20070,Pentose Phosphate Pathway,"H2O[c_PR] + sedoheptulose-1,7-bisphosphate[c_P...",MAM02040c_PR + MAM02883c_PR --> MAM02751c_PR +...,ENSG00000130957 or ENSG00000165140
MAR20089_PR,PR,0.0,1000.0,,MNXR151808,,,SBO:0000176,4.1.1.28,R02701,18533,,MAR20089,Tryptophan metabolism,5-hydroxy-L-tryptophan[c_PR] + H+[c_PR] --> CO...,MAM01107c_PR + MAM02039c_PR --> MAM01596c_PR +...,ENSG00000132437


In [ ]:
# fuse rxn_df and df keep rows sorted based on df 
df_combined = pd.concat([rxn_df, df], axis=1)
# sort df using min_max_loopless['sum']
df_combined = df_combined.reindex(min_max_loopless.sort_values('sum', ascending=False).index)

# save df_combined as excel in results folder / fva_lactate_unlimited
df_combined.to_excel('results/unlimited_lactate/unlimited_lactate.xlsx')
df_combined